# Import Packages and Files

In [8]:
import torch.nn as nn
from functionalities import dataloader as dl
from functionalities import evaluater as ev
from functionalities import filemanager as fm
from functionalities import trainer as tr
from architecture import RotNet as RN

# Load Dataset

In [9]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [10]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 0, 128)

# Initialize Loss Criterion

In [11]:
criterion = nn.CrossEntropyLoss()

# Train RotNet for Rotation Task

In [12]:
# set rot classes
rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']

## 3 Block RotNet

In [13]:
# initialize network
net_block3 = RN.RotNet(num_classes=4, num_conv_block=3, add_avg_pool=False)

In [ ]:
# train network
rot_block3_loss_log, rot_block3_valid_accuracy_log, rot_block3_test_accuracy_log, rot_block3_max_accuracy, \
rot_block3_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block3, 
                                             criterion, trainloader, None, testloader)

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block3_loss_log, block3_valid_accuracy_log, block3_test_accuracy_log, block3_max_accuracy, block3_best_epoch = \
tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block3, criterion, trainloader,
                    None, testloader) 

In [ ]:
# rename files
fm.add_block_to_name(3, [100, 200], )

# Cheating

In [4]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [5]:
# train network with own implemented metric
rot_loss_log, rot_valid_accuracy_log, rot_test_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net, criterion, trainloader, testloader, rot=['90', '180', '270']) 

[1, 60] loss: 1.157
[1, 120] loss: 1.009
[1, 180] loss: 0.918
[1, 240] loss: 0.880
[1, 300] loss: 0.828
[1, 360] loss: 0.789
Epoch: 1 -> Loss: 0.697915673256
Epoch: 1 -> Validation Accuracy: 69.75
[2, 60] loss: 0.735
[2, 120] loss: 0.733
[2, 180] loss: 0.697
[2, 240] loss: 0.695
[2, 300] loss: 0.666
[2, 360] loss: 0.651
Epoch: 2 -> Loss: 0.657023072243
Epoch: 2 -> Validation Accuracy: 72.84
[3, 60] loss: 0.639
[3, 120] loss: 0.612
[3, 180] loss: 0.591
[3, 240] loss: 0.594
[3, 300] loss: 0.590
[3, 360] loss: 0.581
Epoch: 3 -> Loss: 0.434988677502
Epoch: 3 -> Validation Accuracy: 76.805
[4, 60] loss: 0.557
[4, 120] loss: 0.549
[4, 180] loss: 0.545
[4, 240] loss: 0.532
[4, 300] loss: 0.526
[4, 360] loss: 0.521
Epoch: 4 -> Loss: 0.609776377678
Epoch: 4 -> Validation Accuracy: 77.6375
[5, 60] loss: 0.502
[5, 120] loss: 0.521
[5, 180] loss: 0.486
[5, 240] loss: 0.508
[5, 300] loss: 0.497
[5, 360] loss: 0.484
Epoch: 5 -> Loss: 0.474913060665
Epoch: 5 -> Validation Accuracy: 81.0625
[6, 60] lo

[41, 180] loss: 0.308
[41, 240] loss: 0.314
[41, 300] loss: 0.314
[41, 360] loss: 0.317
Epoch: 41 -> Loss: 0.240083023906
Epoch: 41 -> Validation Accuracy: 87.4175
[42, 60] loss: 0.303
[42, 120] loss: 0.293
[42, 180] loss: 0.311
[42, 240] loss: 0.311
[42, 300] loss: 0.323
[42, 360] loss: 0.309
Epoch: 42 -> Loss: 0.285933732986
Epoch: 42 -> Validation Accuracy: 86.9225
[43, 60] loss: 0.305
[43, 120] loss: 0.299
[43, 180] loss: 0.294
[43, 240] loss: 0.313
[43, 300] loss: 0.304
[43, 360] loss: 0.314
Epoch: 43 -> Loss: 0.264206469059
Epoch: 43 -> Validation Accuracy: 87.7375
[44, 60] loss: 0.294
[44, 120] loss: 0.303
[44, 180] loss: 0.299
[44, 240] loss: 0.309
[44, 300] loss: 0.312
[44, 360] loss: 0.315
Epoch: 44 -> Loss: 0.396771132946
Epoch: 44 -> Validation Accuracy: 86.7275
[45, 60] loss: 0.289
[45, 120] loss: 0.296
[45, 180] loss: 0.312
[45, 240] loss: 0.296
[45, 300] loss: 0.324
[45, 360] loss: 0.311
Epoch: 45 -> Loss: 0.390113323927
Epoch: 45 -> Validation Accuracy: 86.945
[46, 60] 

[81, 60] loss: 0.131
[81, 120] loss: 0.139
[81, 180] loss: 0.146
[81, 240] loss: 0.139
[81, 300] loss: 0.140
[81, 360] loss: 0.154
Epoch: 81 -> Loss: 0.26465472579
Epoch: 81 -> Validation Accuracy: 90.195
[82, 60] loss: 0.134
[82, 120] loss: 0.147
[82, 180] loss: 0.144
[82, 240] loss: 0.141
[82, 300] loss: 0.145
[82, 360] loss: 0.147
Epoch: 82 -> Loss: 0.0846824422479
Epoch: 82 -> Validation Accuracy: 90.1325
[83, 60] loss: 0.130
[83, 120] loss: 0.132
[83, 180] loss: 0.140
[83, 240] loss: 0.148
[83, 300] loss: 0.151
[83, 360] loss: 0.161
Epoch: 83 -> Loss: 0.107063688338
Epoch: 83 -> Validation Accuracy: 90.275
[84, 60] loss: 0.135
[84, 120] loss: 0.146
[84, 180] loss: 0.143
[84, 240] loss: 0.150
[84, 300] loss: 0.153
[84, 360] loss: 0.138
Epoch: 84 -> Loss: 0.166196703911
Epoch: 84 -> Validation Accuracy: 90.865
[85, 60] loss: 0.127
[85, 120] loss: 0.130
[85, 180] loss: 0.145
[85, 240] loss: 0.150
[85, 300] loss: 0.143
[85, 360] loss: 0.146
Epoch: 85 -> Loss: 0.115848943591
Epoch: 85 

[120, 120] loss: 0.119
[120, 180] loss: 0.123
[120, 240] loss: 0.129
[120, 300] loss: 0.136
[120, 360] loss: 0.139
Epoch: 120 -> Loss: 0.118816539645
Epoch: 120 -> Validation Accuracy: 90.16
[121, 60] loss: 0.095
[121, 120] loss: 0.074
[121, 180] loss: 0.066
[121, 240] loss: 0.064
[121, 300] loss: 0.069
[121, 360] loss: 0.063
Epoch: 121 -> Loss: 0.0384818091989
Epoch: 121 -> Validation Accuracy: 92.39
[122, 60] loss: 0.055
[122, 120] loss: 0.053
[122, 180] loss: 0.048
[122, 240] loss: 0.055
[122, 300] loss: 0.053
[122, 360] loss: 0.048
Epoch: 122 -> Loss: 0.0428778603673
Epoch: 122 -> Validation Accuracy: 92.36
[123, 60] loss: 0.047
[123, 120] loss: 0.045
[123, 180] loss: 0.049
[123, 240] loss: 0.045
[123, 300] loss: 0.050
[123, 360] loss: 0.049
Epoch: 123 -> Loss: 0.0411820337176
Epoch: 123 -> Validation Accuracy: 92.28
[124, 60] loss: 0.043
[124, 120] loss: 0.040
[124, 180] loss: 0.042
[124, 240] loss: 0.046
[124, 300] loss: 0.043
[124, 360] loss: 0.047
Epoch: 124 -> Loss: 0.04493727

[158, 180] loss: 0.026
[158, 240] loss: 0.027
[158, 300] loss: 0.025
[158, 360] loss: 0.028
Epoch: 158 -> Loss: 0.0363478139043
Epoch: 158 -> Validation Accuracy: 91.375
[159, 60] loss: 0.023
[159, 120] loss: 0.024
[159, 180] loss: 0.024
[159, 240] loss: 0.027
[159, 300] loss: 0.026
[159, 360] loss: 0.026
Epoch: 159 -> Loss: 0.0388043783605
Epoch: 159 -> Validation Accuracy: 91.63
[160, 60] loss: 0.023
[160, 120] loss: 0.026
[160, 180] loss: 0.026
[160, 240] loss: 0.028
[160, 300] loss: 0.029
[160, 360] loss: 0.027
Epoch: 160 -> Loss: 0.012322044
Epoch: 160 -> Validation Accuracy: 91.8825
[161, 60] loss: 0.020
[161, 120] loss: 0.016
[161, 180] loss: 0.015
[161, 240] loss: 0.014
[161, 300] loss: 0.014
[161, 360] loss: 0.013
Epoch: 161 -> Loss: 0.0114490520209
Epoch: 161 -> Validation Accuracy: 92.135
[162, 60] loss: 0.012
[162, 120] loss: 0.013
[162, 180] loss: 0.012
[162, 240] loss: 0.011
[162, 300] loss: 0.011
[162, 360] loss: 0.012
Epoch: 162 -> Loss: 0.00763732194901
Epoch: 162 -> V

[196, 180] loss: 0.005
[196, 240] loss: 0.005
[196, 300] loss: 0.005
[196, 360] loss: 0.005
Epoch: 196 -> Loss: 0.00783396046609
Epoch: 196 -> Validation Accuracy: 92.2625
[197, 60] loss: 0.005
[197, 120] loss: 0.005
[197, 180] loss: 0.005
[197, 240] loss: 0.006
[197, 300] loss: 0.005
[197, 360] loss: 0.006
Epoch: 197 -> Loss: 0.00178200227674
Epoch: 197 -> Validation Accuracy: 92.32
[198, 60] loss: 0.005
[198, 120] loss: 0.006
[198, 180] loss: 0.005
[198, 240] loss: 0.005
[198, 300] loss: 0.005
[198, 360] loss: 0.005
Epoch: 198 -> Loss: 0.00199384381995
Epoch: 198 -> Validation Accuracy: 92.265
[199, 60] loss: 0.005
[199, 120] loss: 0.005
[199, 180] loss: 0.005
[199, 240] loss: 0.005
[199, 300] loss: 0.005
[199, 360] loss: 0.005
Epoch: 199 -> Loss: 0.0138222128153
Epoch: 199 -> Validation Accuracy: 92.25
[200, 60] loss: 0.005
[200, 120] loss: 0.005
[200, 180] loss: 0.005
[200, 240] loss: 0.005
[200, 300] loss: 0.005
[200, 360] loss: 0.005
Epoch: 200 -> Loss: 0.00204133382067
Epoch: 20

In [6]:
# test RotNet
#net = fm.load_net('RotNet_rotation_159_best')
rot_accuracy = ev.get_accuracy(testloader, net, ['90', '180', '270'])
class_accuracy = ev.get_class_accuracy(4, testloader, net, rot_classes, ['90', '180', '270'])

Test Accuracy:  92.343 %
Test Accuracy of original :  92.480 %
Test Accuracy of 90 rotation :  92.080 %
Test Accuracy of 180 rotation :  92.300 %
Test Accuracy of 270 rotation :  92.510 %


In [7]:
#net = fm.load_net('RotNet_rotation_200')
paper_rot_accuracy = ev.get_accuracy(testloader, net, ['90', '180', '270'], use_paper_metric=True)

Test Accuracy:  92.410 %


In [8]:
loss_log, valid_accuracy_log, test_accuracy_log, max_accuracy, best_epoch = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net, criterion, trainloader, testloader) #[20, 40, 45, 50]

[1, 60] loss: 2.231
[1, 120] loss: 1.268
[1, 180] loss: 1.157
[1, 240] loss: 1.067
[1, 300] loss: 1.022
[1, 360] loss: 0.998
Epoch: 1 -> Loss: 1.12693870068
Epoch: 1 -> Validation Accuracy: 67.49
[2, 60] loss: 0.934
[2, 120] loss: 0.919
[2, 180] loss: 0.905
[2, 240] loss: 0.884
[2, 300] loss: 0.886
[2, 360] loss: 0.871
Epoch: 2 -> Loss: 0.879200458527
Epoch: 2 -> Validation Accuracy: 71.94
[3, 60] loss: 0.829
[3, 120] loss: 0.837
[3, 180] loss: 0.847
[3, 240] loss: 0.809
[3, 300] loss: 0.815
[3, 360] loss: 0.788
Epoch: 3 -> Loss: 0.689446568489
Epoch: 3 -> Validation Accuracy: 73.3
[4, 60] loss: 0.769
[4, 120] loss: 0.794
[4, 180] loss: 0.751
[4, 240] loss: 0.755
[4, 300] loss: 0.761
[4, 360] loss: 0.750
Epoch: 4 -> Loss: 0.681841433048
Epoch: 4 -> Validation Accuracy: 75.53
[5, 60] loss: 0.723
[5, 120] loss: 0.745
[5, 180] loss: 0.730
[5, 240] loss: 0.710
[5, 300] loss: 0.721
[5, 360] loss: 0.749
Epoch: 5 -> Loss: 0.666413664818
Epoch: 5 -> Validation Accuracy: 76.51
[6, 60] loss: 0.6

[41, 300] loss: 0.341
[41, 360] loss: 0.334
Epoch: 41 -> Loss: 0.38911730051
Epoch: 41 -> Validation Accuracy: 82.27
[42, 60] loss: 0.335
[42, 120] loss: 0.321
[42, 180] loss: 0.332
[42, 240] loss: 0.334
[42, 300] loss: 0.315
[42, 360] loss: 0.327
Epoch: 42 -> Loss: 0.551990628242
Epoch: 42 -> Validation Accuracy: 82.69
[43, 60] loss: 0.305
[43, 120] loss: 0.307
[43, 180] loss: 0.314
[43, 240] loss: 0.321
[43, 300] loss: 0.318
[43, 360] loss: 0.304
Epoch: 43 -> Loss: 0.367444097996
Epoch: 43 -> Validation Accuracy: 82.51
[44, 60] loss: 0.300
[44, 120] loss: 0.304
[44, 180] loss: 0.316
[44, 240] loss: 0.309
[44, 300] loss: 0.309
[44, 360] loss: 0.317
Epoch: 44 -> Loss: 0.316638618708
Epoch: 44 -> Validation Accuracy: 82.72
[45, 60] loss: 0.304
[45, 120] loss: 0.287
[45, 180] loss: 0.296
[45, 240] loss: 0.316
[45, 300] loss: 0.300
[45, 360] loss: 0.303
Epoch: 45 -> Loss: 0.280343532562
Epoch: 45 -> Validation Accuracy: 82.72
[46, 60] loss: 0.299
[46, 120] loss: 0.290
[46, 180] loss: 0.28

[81, 300] loss: 0.250
[81, 360] loss: 0.233
Epoch: 81 -> Loss: 0.287597030401
Epoch: 81 -> Validation Accuracy: 83.0
[82, 60] loss: 0.249
[82, 120] loss: 0.246
[82, 180] loss: 0.244
[82, 240] loss: 0.245
[82, 300] loss: 0.247
[82, 360] loss: 0.236
Epoch: 82 -> Loss: 0.215925693512
Epoch: 82 -> Validation Accuracy: 83.11
[83, 60] loss: 0.238
[83, 120] loss: 0.231
[83, 180] loss: 0.243
[83, 240] loss: 0.245
[83, 300] loss: 0.259
[83, 360] loss: 0.240
Epoch: 83 -> Loss: 0.23143453896
Epoch: 83 -> Validation Accuracy: 83.09
[84, 60] loss: 0.240
[84, 120] loss: 0.236
[84, 180] loss: 0.252
[84, 240] loss: 0.238
[84, 300] loss: 0.249
[84, 360] loss: 0.258
Epoch: 84 -> Loss: 0.196297630668
Epoch: 84 -> Validation Accuracy: 82.98
[85, 60] loss: 0.254
[85, 120] loss: 0.239
[85, 180] loss: 0.238
[85, 240] loss: 0.240
[85, 300] loss: 0.249
[85, 360] loss: 0.226
Epoch: 85 -> Loss: 0.206151202321
Epoch: 85 -> Validation Accuracy: 83.08
[86, 60] loss: 0.247
[86, 120] loss: 0.224
[86, 180] loss: 0.238

[21, 300] loss: 0.276
[21, 360] loss: 0.278
Epoch: 21 -> Loss: 0.330422818661
Epoch: 21 -> Validation Accuracy: 85.57
[22, 60] loss: 0.265
[22, 120] loss: 0.276
[22, 180] loss: 0.258
[22, 240] loss: 0.269
[22, 300] loss: 0.274
[22, 360] loss: 0.275
Epoch: 22 -> Loss: 0.28525146842
Epoch: 22 -> Validation Accuracy: 85.7
[23, 60] loss: 0.240
[23, 120] loss: 0.241
[23, 180] loss: 0.258
[23, 240] loss: 0.240
[23, 300] loss: 0.254
[23, 360] loss: 0.244
Epoch: 23 -> Loss: 0.325827330351
Epoch: 23 -> Validation Accuracy: 86.22
[24, 60] loss: 0.235
[24, 120] loss: 0.226
[24, 180] loss: 0.234
[24, 240] loss: 0.228
[24, 300] loss: 0.250
[24, 360] loss: 0.234
Epoch: 24 -> Loss: 0.167077973485
Epoch: 24 -> Validation Accuracy: 85.85
[25, 60] loss: 0.230
[25, 120] loss: 0.210
[25, 180] loss: 0.241
[25, 240] loss: 0.236
[25, 300] loss: 0.227
[25, 360] loss: 0.227
Epoch: 25 -> Loss: 0.262460649014
Epoch: 25 -> Validation Accuracy: 85.95
[26, 60] loss: 0.211
[26, 120] loss: 0.214
[26, 180] loss: 0.210

[61, 300] loss: 0.095
[61, 360] loss: 0.101
Epoch: 61 -> Loss: 0.162266105413
Epoch: 61 -> Validation Accuracy: 87.04
[62, 60] loss: 0.106
[62, 120] loss: 0.102
[62, 180] loss: 0.103
[62, 240] loss: 0.098
[62, 300] loss: 0.098
[62, 360] loss: 0.104
Epoch: 62 -> Loss: 0.109789177775
Epoch: 62 -> Validation Accuracy: 86.95
[63, 60] loss: 0.096
[63, 120] loss: 0.100
[63, 180] loss: 0.100
[63, 240] loss: 0.100
[63, 300] loss: 0.100
[63, 360] loss: 0.098
Epoch: 63 -> Loss: 0.096001714468
Epoch: 63 -> Validation Accuracy: 86.9
[64, 60] loss: 0.096
[64, 120] loss: 0.098
[64, 180] loss: 0.102
[64, 240] loss: 0.094
[64, 300] loss: 0.098
[64, 360] loss: 0.101
Epoch: 64 -> Loss: 0.149449497461
Epoch: 64 -> Validation Accuracy: 86.88
[65, 60] loss: 0.101
[65, 120] loss: 0.089
[65, 180] loss: 0.099
[65, 240] loss: 0.103
[65, 300] loss: 0.099
[65, 360] loss: 0.092
Epoch: 65 -> Loss: 0.117008969188
Epoch: 65 -> Validation Accuracy: 86.93
[66, 60] loss: 0.097
[66, 120] loss: 0.098
[66, 180] loss: 0.10

[1, 60] loss: 1.670
[1, 120] loss: 0.907
[1, 180] loss: 0.829
[1, 240] loss: 0.797
[1, 300] loss: 0.778
[1, 360] loss: 0.736
Epoch: 1 -> Loss: 0.867527484894
Epoch: 1 -> Validation Accuracy: 73.83
[2, 60] loss: 0.715
[2, 120] loss: 0.696
[2, 180] loss: 0.672
[2, 240] loss: 0.666
[2, 300] loss: 0.658
[2, 360] loss: 0.666
Epoch: 2 -> Loss: 0.736922860146
Epoch: 2 -> Validation Accuracy: 74.83
[3, 60] loss: 0.637
[3, 120] loss: 0.648
[3, 180] loss: 0.608
[3, 240] loss: 0.642
[3, 300] loss: 0.625
[3, 360] loss: 0.623
Epoch: 3 -> Loss: 0.802419662476
Epoch: 3 -> Validation Accuracy: 75.59
[4, 60] loss: 0.591
[4, 120] loss: 0.561
[4, 180] loss: 0.586
[4, 240] loss: 0.598
[4, 300] loss: 0.609
[4, 360] loss: 0.607
Epoch: 4 -> Loss: 0.639377474785
Epoch: 4 -> Validation Accuracy: 76.25
[5, 60] loss: 0.566
[5, 120] loss: 0.577
[5, 180] loss: 0.575
[5, 240] loss: 0.571
[5, 300] loss: 0.582
[5, 360] loss: 0.583
Epoch: 5 -> Loss: 0.533836185932
Epoch: 5 -> Validation Accuracy: 77.4
[6, 60] loss: 0.

[41, 300] loss: 0.308
[41, 360] loss: 0.327
Epoch: 41 -> Loss: 0.258478820324
Epoch: 41 -> Validation Accuracy: 81.67
[42, 60] loss: 0.291
[42, 120] loss: 0.304
[42, 180] loss: 0.304
[42, 240] loss: 0.313
[42, 300] loss: 0.280
[42, 360] loss: 0.302
Epoch: 42 -> Loss: 0.435032308102
Epoch: 42 -> Validation Accuracy: 81.56
[43, 60] loss: 0.290
[43, 120] loss: 0.287
[43, 180] loss: 0.295
[43, 240] loss: 0.286
[43, 300] loss: 0.292
[43, 360] loss: 0.292
Epoch: 43 -> Loss: 0.459386020899
Epoch: 43 -> Validation Accuracy: 81.56
[44, 60] loss: 0.292
[44, 120] loss: 0.295
[44, 180] loss: 0.290
[44, 240] loss: 0.282
[44, 300] loss: 0.286
[44, 360] loss: 0.275
Epoch: 44 -> Loss: 0.389640599489
Epoch: 44 -> Validation Accuracy: 81.44
[45, 60] loss: 0.272
[45, 120] loss: 0.298
[45, 180] loss: 0.273
[45, 240] loss: 0.288
[45, 300] loss: 0.276
[45, 360] loss: 0.282
Epoch: 45 -> Loss: 0.217036247253
Epoch: 45 -> Validation Accuracy: 81.61
[46, 60] loss: 0.268
[46, 120] loss: 0.270
[46, 180] loss: 0.2

[81, 360] loss: 0.230
Epoch: 81 -> Loss: 0.299762964249
Epoch: 81 -> Validation Accuracy: 81.85
[82, 60] loss: 0.242
[82, 120] loss: 0.236
[82, 180] loss: 0.225
[82, 240] loss: 0.221
[82, 300] loss: 0.231
[82, 360] loss: 0.241
Epoch: 82 -> Loss: 0.21850939095
Epoch: 82 -> Validation Accuracy: 81.86
[83, 60] loss: 0.231
[83, 120] loss: 0.232
[83, 180] loss: 0.226
[83, 240] loss: 0.218
[83, 300] loss: 0.227
[83, 360] loss: 0.231
Epoch: 83 -> Loss: 0.333632320166
Epoch: 83 -> Validation Accuracy: 81.73
[84, 60] loss: 0.231
[84, 120] loss: 0.221
[84, 180] loss: 0.225
[84, 240] loss: 0.216
[84, 300] loss: 0.240
[84, 360] loss: 0.229
Epoch: 84 -> Loss: 0.318319708109
Epoch: 84 -> Validation Accuracy: 81.84
[85, 60] loss: 0.236
[85, 120] loss: 0.223
[85, 180] loss: 0.227
[85, 240] loss: 0.239
[85, 300] loss: 0.222
[85, 360] loss: 0.221
Epoch: 85 -> Loss: 0.378499001265
Epoch: 85 -> Validation Accuracy: 81.74
[86, 60] loss: 0.221
[86, 120] loss: 0.229
[86, 180] loss: 0.217
[86, 240] loss: 0.22

Epoch: 21 -> Loss: 1.6341354847
Epoch: 21 -> Validation Accuracy: 40.97
[22, 60] loss: 1.552
[22, 120] loss: 1.540
[22, 180] loss: 1.543
[22, 240] loss: 1.554
[22, 300] loss: 1.542
[22, 360] loss: 1.528
Epoch: 22 -> Loss: 1.56702494621
Epoch: 22 -> Validation Accuracy: 41.24
[23, 60] loss: 1.531
[23, 120] loss: 1.527
[23, 180] loss: 1.534
[23, 240] loss: 1.536
[23, 300] loss: 1.543
[23, 360] loss: 1.541
Epoch: 23 -> Loss: 1.50636506081
Epoch: 23 -> Validation Accuracy: 41.09
[24, 60] loss: 1.520
[24, 120] loss: 1.492
[24, 180] loss: 1.521
[24, 240] loss: 1.541
[24, 300] loss: 1.517
[24, 360] loss: 1.543
Epoch: 24 -> Loss: 1.52132213116
Epoch: 24 -> Validation Accuracy: 40.51
[25, 60] loss: 1.510
[25, 120] loss: 1.512
[25, 180] loss: 1.526
[25, 240] loss: 1.535
[25, 300] loss: 1.513
[25, 360] loss: 1.539
Epoch: 25 -> Loss: 1.41445422173
Epoch: 25 -> Validation Accuracy: 41.45
[26, 60] loss: 1.539
[26, 120] loss: 1.506
[26, 180] loss: 1.510
[26, 240] loss: 1.513
[26, 300] loss: 1.527
[26

[62, 60] loss: 1.380
[62, 120] loss: 1.401
[62, 180] loss: 1.379
[62, 240] loss: 1.395
[62, 300] loss: 1.401
[62, 360] loss: 1.394
Epoch: 62 -> Loss: 1.39932370186
Epoch: 62 -> Validation Accuracy: 43.84
[63, 60] loss: 1.378
[63, 120] loss: 1.377
[63, 180] loss: 1.397
[63, 240] loss: 1.399
[63, 300] loss: 1.403
[63, 360] loss: 1.389
Epoch: 63 -> Loss: 1.32465732098
Epoch: 63 -> Validation Accuracy: 43.84
[64, 60] loss: 1.360
[64, 120] loss: 1.384
[64, 180] loss: 1.394
[64, 240] loss: 1.390
[64, 300] loss: 1.377
[64, 360] loss: 1.392
Epoch: 64 -> Loss: 1.37368476391
Epoch: 64 -> Validation Accuracy: 44.14
[65, 60] loss: 1.408
[65, 120] loss: 1.375
[65, 180] loss: 1.387
[65, 240] loss: 1.409
[65, 300] loss: 1.388
[65, 360] loss: 1.378
Epoch: 65 -> Loss: 1.41299116611
Epoch: 65 -> Validation Accuracy: 44.09
[66, 60] loss: 1.368
[66, 120] loss: 1.387
[66, 180] loss: 1.382
[66, 240] loss: 1.403
[66, 300] loss: 1.385
[66, 360] loss: 1.407
Epoch: 66 -> Loss: 1.5088186264
Epoch: 66 -> Validati

In [9]:
clf_1 = fm.load_net('classifier_block_1_epoch_100')
clf_2 = fm.load_net('classifier_block_2_epoch_100')
clf_3 = fm.load_net('classifier_block_3_epoch_100')
clf_4 = fm.load_net('classifier_block_4_epoch_100')

In [10]:
paper_accuracy_1 = ev.get_accuracy(testloader, net, classifier=clf_1, conv_block_num=1, use_paper_metric=True)
paper_accuracy_2 = ev.get_accuracy(testloader, net, classifier=clf_2, conv_block_num=2, use_paper_metric=True)
paper_accuracy_3 = ev.get_accuracy(testloader, net, classifier=clf_3, conv_block_num=3, use_paper_metric=True)
paper_accuracy_4 = ev.get_accuracy(testloader, net, classifier=clf_4, conv_block_num=4, use_paper_metric=True)

Test Accuracy:  83.109 %
Test Accuracy:  86.531 %
Test Accuracy:  81.655 %
Test Accuracy:  44.581 %
